In [1]:
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from keras.layers import GlobalAveragePooling2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.layers import BatchNormalization
import os 
import pandas as pd
import plotly.graph_objs as go
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
import plotly.express as px
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [3]:
!pip install opendatasets
import opendatasets as od
od.download('https://www.kaggle.com/datasets/raghavrpotdar/fresh-and-stale-images-of-fruits-and-vegetables')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gaurav065
Your Kaggle Key: ··········


100%|██████████| 1.41G/1.41G [01:06<00:00, 22.9MB/s]


In [4]:
train ='/content/fresh-and-stale-images-of-fruits-and-vegetables'
# validation_dir = '/content/fruits-fresh-and-rotten-for-classification/dataset/train'

In [5]:
import glob
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count

In [6]:
train_samples =get_files(train)
num_classes=len(glob.glob(train+"/*"))
# test_samples=get_files(validation_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
# print(test_samples,"Test images")

13 Classes
14682 Train images


In [10]:
from keras.applications.mobilenet import preprocess_input
import tensorflow as tf
training=tf.keras.preprocessing.image.ImageDataGenerator(
    zca_epsilon=1e-06,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.30
).flow_from_directory(train,batch_size=16,target_size=(224,224),subset="training")

validing=tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode="nearest",
    horizontal_flip=True,
    vertical_flip=True,
    preprocessing_function=preprocess_input,
    validation_split=0.30
).flow_from_directory(train,batch_size=16,target_size=(224,224),subset='validation',shuffle=True)

Found 10282 images belonging to 12 classes.
Found 4400 images belonging to 12 classes.


In [16]:
from keras import Model
from keras import optimizers
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import MaxPooling2D, Conv2D, Dense, Flatten, Dropout

model = Sequential()
model.add(Conv2D(filters=32,kernel_size=3, padding = 'same', activation='relu',input_shape = (224, 224, 3)))
model.add(Conv2D(filters=32, kernel_size=3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#model.add(Dropout(0.25))
model.add(Conv2D(filters=64,kernel_size=3, padding = 'same', activation='relu'))
model.add(Conv2D(filters=64, kernel_size=3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#model.add(Dropout(0.25))

model.add(Conv2D(filters=128,kernel_size=3, padding = 'same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size=3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# model.add(Dropout(0.25))

model.add(Conv2D(filters=256,kernel_size=3, padding = 'same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# model.add(Dropout(0.25))

model.add(Conv2D(filters=256,kernel_size=3, padding = 'same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size=3, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
# model.add(Dropout(0.25))



model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.4))
# model.add(Dropout(0.5))
model.add(Dense(12, activation='softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 conv2d_11 (Conv2D)          (None, 224, 224, 32)      9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 112, 112, 32)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 112, 112, 32)      0         
                                                                 
 conv2d_12 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 conv2d_13 (Conv2D)          (None, 112, 112, 64)      36928     
                                                      

In [17]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
history = model.fit(training,validation_data=validing,epochs=100,shuffle=True,callbacks=early_stopping_callback)

Epoch 1/100
643/643 [==============================] - 218s 340ms/step - loss: 0.9753 - accuracy: 0.6502 - val_loss: 0.9234 - val_accuracy: 0.6348
Epoch 2/100
643/643 [==============================] - 218s 339ms/step - loss: 0.6676 - accuracy: 0.7618 - val_loss: 0.7617 - val_accuracy: 0.7123
Epoch 3/100
643/643 [==============================] - 220s 342ms/step - loss: 0.5054 - accuracy: 0.8156 - val_loss: 0.7001 - val_accuracy: 0.7148
Epoch 4/100
643/643 [==============================] - 219s 341ms/step - loss: 0.4106 - accuracy: 0.8572 - val_loss: 0.6229 - val_accuracy: 0.7575
Epoch 5/100
643/643 [==============================] - 219s 340ms/step - loss: 0.3870 - accuracy: 0.8639 - val_loss: 0.7012 - val_accuracy: 0.7423
Epoch 6/100
643/643 [==============================] - 232s 360ms/step - loss: 0.3572 - accuracy: 0.8718 - val_loss: 0.6039 - val_accuracy: 0.7718
Epoch 7/100
643/643 [==============================] - 219s 341ms/step - loss: 0.3219 - accuracy: 0.8818 - val_loss: 0

In [ ]:
model.save('cnn_food_model.h5')

In [ ]:
accuracy_values = history.history['accuracy']
val_accuracy_values = history.history['val_accuracy']
loss_values = history.history['loss']
val_loss_values = history.history['val_loss']

# Plot the graph
plt.figure(figsize=(10, 5))  # adjust the figure size as needed

# Plot accuracy values
plt.subplot(1, 2, 1)  # create a subplot for accuracy
plt.plot(accuracy_values, label='Train Accuracy')
plt.plot(val_accuracy_values, label='Validation Accuracy')
plt.title('Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss values
plt.subplot(1, 2, 2)  # create a subplot for loss
plt.plot(loss_values, label='Train Loss')
plt.plot(val_loss_values, label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()  # adjust the spacing between subplots if needed
plt.show()

In [ ]:
model.evaluate(validing, verbose=1)

341/341 [==============================] - 65s 190ms/step - loss: 2.3252 - accuracy: 0.7353


[2.3252334594726562, 0.735253632068634]